# Librerias

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

# Token

# Coloca tu token personal de Banxico
BANXICO_TOKEN = "caa9ce3af73a7040c54f64908e2af98fd965b7c816828c6d90836ff2dd38a9d2"

serie_id = "SF43783"  # ID de la TIIE a 28 días
url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_id}/datos"

headers = {"Bmx-Token": BANXICO_TOKEN}
response = requests.get(url, headers=headers)
data = response.json()

# Convertir a DataFrame
datos = data['bmx']['series'][0]['datos']
df = pd.DataFrame(datos)
df['fecha'] = pd.to_datetime(df['fecha'])
df['dato'] = pd.to_numeric(df['dato'], errors='coerce')
df = df.set_index('fecha').sort_index()